# Introduction
[Chainer](http://chainer.org/) とはニューラルネットの実装を簡単にしたフレームワークです。

* 今回は言語の分野でニューラルネットを適用してみました。

![](./pictures/Chainer.jpg)

* 今回は言語モデルを作成していただきます。


言語モデルとはある単語が来たときに次の単語に何が来やすいかを予測するものです。

言語モデルにはいくつか種類があるのでここでも紹介しておきます。

* n-グラム言語モデル
 * 単語の数を単純に数え挙げて作成されるモデル。考え方としてはデータにおけるある単語の頻度に近い
* ニューラル言語モデル
 * 単語の辞書ベクトルを潜在空間ベクトルに落とし込み、ニューラルネットで次の文字を学習させる手法

* リカレントニューラル言語モデル
 * 基本的なアルゴリズムはニューラル言語モデルと同一だが過去に使用した単語を入力に加えることによって文脈を考慮した言語モデルの学習が可能となる。ニューラル言語モデルとは異なり、より古い情報も取得可能

以下では、このChainerを利用しデータを準備するところから実際に言語モデルを構築し学習・評価を行うまでの手順を解説します。

1. [各種ライブラリ導入](#各種ライブラリ導入) 
2. [初期設定](#初期設定) 
3. [データ入力](#データ入力)
4. [リカレントニューラル言語モデル設定](#リカレントニューラル言語モデル設定(ハンズオン）) 
5. [学習を始める前の設定](#学習を始める前の設定)
6. [パラメータ更新方法](#パラメータ更新方法（ミニバッチ学習）)
7. [言語の予測](#言語の予測)

もしGPUを使用したい方は、以下にまとめてあるのでご参考ください。

[Chainer を用いてリカレントニューラル言語モデル作成のサンプルコードを解説してみた](http://qiita.com/GushiSnow/private/b34da4962dd930d1487a)



## 各種ライブラリ導入

Chainerの言語処理では多数のライブラリを導入します。



In [2]:
import time
import math
import sys
import pickle
import copy
import os
import re

import numpy as np
from chainer import cuda, Variable, FunctionSet, optimizers
import chainer.functions as F

`導入するライブラリの代表例は下記です。

* `numpy`: 行列計算などの複雑な計算を行なうライブラリ
* `chainer`: Chainerの導入


## 初期設定

下記を設定しています。
* 学習回数：n_epochs
* ニューラルネットのユニット数：n_units
* 確率的勾配法に使用するデータの数：batchsize
* 学習に使用する文字列の長さ：bprop_len
* 勾配法で使用する敷居値：grad_clip
* 学習データの格納場所：data_dir
* モデルの出力場所：checkpoint_dir



In [3]:
#-------------Explain7 in the Qiita-------------
n_epochs    = 5
n_units     = 50
batchsize   = 100
bprop_len   = 10
grad_clip   = 1
data_dir = "data_hands_on"
checkpoint_dir = "cv"
#-------------Explain7 in the Qiita-------------

## データ入力

学習用にダウンロードしたファイルをプログラムに読ませる処理を関数化しています
文字列の場合は通常のデータと異なり、数字ベクトル化する必要があります。

* 学習データをバイナリ形式で読み込んでいます。
* 文字データを確保するための行列を定義しています。
* データは単語をキー、語彙数の連番idを値とした辞書データにして行列データセットに登録しています。

学習データ、単語の長さ、語彙数を取得しています。
上記をそれぞれ行列データとして保持しています。


In [19]:
# input data
#-------------Explain1 in the Qiita-------------
def source_to_words(source):
    line = source.replace("¥n", " ").replace("¥t", " ")
    for spacer in ["(", ")", "{", "}", "[", "]", ","]:
        line = line.replace(spacer, " " + spacer + " ")
    
    words = [w.strip() for w in line.split()]
    return words

def load_data():
    vocab = {}
    print ('%s/angular.js'% data_dir)
    source = open('%s/angular_full_remake.js' % data_dir, 'r').read()
    words = source_to_words(source)
    dataset = np.ndarray((len(words),), dtype=np.int32)
    for i, word in enumerate(words):
        if word not in vocab:
            vocab[word] = len(vocab)
        dataset[i] = vocab[word]
    print('corpus length:', len(words))
    print('vocab size:', len(vocab))
    return dataset, words, vocab
#-------------Explain1 in the Qiita-------------

if not os.path.exists(checkpoint_dir):
    os.mkdir(checkpoint_dir)

train_data, words, vocab = load_data()
print(vocab)

data_hands_on/angular.js
corpus length: 665712
vocab size: 9009
{'objType': 1665, 'me</summary>': 6319, '.ng-cloak': 8998, '.animate-if.ng-leave.ng-leave-active': 7530, 'xsrfValue': 3502, 'name="http-hello.html">': 3370, "'</tr></tbody></table>'": 1291, 'this.$setViewValue': 7865, 'attribute': 2539, 'window.clearTimeout': 2133, 'window.location': 2128, 'ast.watchId': 4544, 'definition': 2389, 'prop:': 1511, 'changes.': 2540, 'salutationInput.sendKeys': 7094, '_': 1257, '#ebccd1;': 752, 'format': 5782, '@kind': 217, "length'": 7016, 'tzMin': 5987, 'ev.type;': 6826, 'hasApply': 4041, 'lastCookies;': 5542, '"Jan"': 8927, 'map;': 6870, 'ng\\\\:cloak': 8994, 'Security': 7410, 'ctrl.$pending': 8013, '"."': 8964, 'applyAsyncQueue.shift': 5216, 'NG_ENABLE_DEBUG_INFO': 812, 'padding:0': 8437, '$parseOptions': 4752, 'invokeArgs': 1784, 'result;': 1526, "'cpi'": 460, '.not.': 7270, 'controllerResult;': 2898, 'getTypeForFilter': 5685, 'person1': 8328, 'this.nextUid': 1668, "'1':": 8327, 'BOOLEAN_A

## リカレントニューラル言語モデル設定(ハンズオン）

RNNLM(リカレントニューラル言語モデルの設定を行っています）
この部分で自由にモデルを変えることが出来ます。
この部分でリカレントニューラル言語モデル独特の特徴を把握してもらうことが目的です。

* EmbedIDで行列変換を行い、疎なベクトルを密なベクトルに変換しています。辞書データを、入力ユニット数分のデータに変換する処理（潜在ベクトル空間への変換）を行っています。
* 出力が4倍の理由は入力層、入力制限層、出力制限層、忘却層をLSTMでは入力に使用するためです。LSTMの魔法の工夫について知りたい方は下記をご覧下さい。
http://www.slideshare.net/nishio/long-shortterm-memory
* `h1_in   = self.l1_x(F.dropout(h0, ratio=dropout_ratio, train=train)) + self.l1_h(state['h1'])`は隠れ層に前回保持した隠れ層の状態を入力することによってLSTMを実現しています。
* `F.dropout`は過去の情報を保持しながらどれだけのdropoutでユニットを削るかを表しています。これにより過学習するのを抑えています。
 Drop outについては下記をご覧下さい。

 http://olanleed.hatenablog.com/entry/2013/12/03/010945

* `c1, h1  = F.lstm(state['c1'], h1_in)`はlstmと呼ばれる魔法の工夫によってリカレントニューラルネットがメモリ破綻を起こさずにいい感じで学習するための工夫です。詳しく知りたい人は下記をご覧下さい。
* `return state, F.softmax_cross_entropy(y, t)`は予測した文字と実際の文字を比較して損失関数を更新している所になります。ソフトマックス関数を使用している理由は出力層の一つ前の層の全入力を考慮して出力を決定できるので一般的に出力層の計算にはソフトマックス関数が使用されます。
* 予測を行なうメソッドも実装しており、入力されたデータ、状態を元に次の文字列と状態を返すような関数になっています。
* モデルの初期化を行なう関数もここで定義しています。

下記をコーディングして下さい！！！！


In [20]:
#-------------Explain2 in the Qiita-------------
class CharRNN(FunctionSet):

    def __init__(self, n_vocab, n_units):
        super(CharRNN, self).__init__(
            embed = F.EmbedID(n_vocab, n_units),
            l1_x = F.Linear(n_units, 4*n_units),
            l1_h = F.Linear(n_units, 4*n_units),
            l2_h = F.Linear(n_units, 4*n_units),
            l2_x = F.Linear(n_units, 4*n_units),
            l3   = F.Linear(n_units, n_vocab),
        )
        for param in self.parameters:
            param[:] = np.random.uniform(-0.08, 0.08, param.shape)

    def forward_one_step(self, x_data, y_data, state, train=True, dropout_ratio=0.5):
        x = Variable(x_data, volatile=not train)
        t = Variable(y_data, volatile=not train)

        h0      = self.embed(x)
        h1_in   = self.l1_x(F.dropout(h0, ratio=dropout_ratio, train=train)) + self.l1_h(state['h1'])
        c1, h1  = F.lstm(state['c1'], h1_in)
        h2_in   = self.l2_x(F.dropout(h1, ratio=dropout_ratio, train=train)) + self.l2_h(state['h2'])
        c2, h2  = F.lstm(state['c2'], h2_in)
        y       = self.l3(F.dropout(h2, ratio=dropout_ratio, train=train))
        state   = {'c1': c1, 'h1': h1, 'c2': c2, 'h2': h2}

        return state, F.softmax_cross_entropy(y, t)

    def predict(self, x_data, state):
        x = Variable(x_data, volatile=True)

        h0      = self.embed(x)
        h1_in   = self.l1_x(h0) + self.l1_h(state['h1'])
        c1, h1  = F.lstm(state['c1'], h1_in)
        h2_in   = self.l2_x(h1) + self.l2_h(state['h2'])
        c2, h2  = F.lstm(state['c2'], h2_in)
        y       = self.l3(h2)
        state   = {'c1': c1, 'h1': h1, 'c2': c2, 'h2': h2}

        return state, F.softmax(y)

def make_initial_state(n_units, batchsize=50, train=True):
    return {name: Variable(np.zeros((batchsize, n_units), dtype=np.float32),
            volatile=not train)
            for name in ('c1', 'h1', 'c2', 'h2')}
#-------------Explain2 in the Qiita-------------

RNNLM(リカレントニューラル言語モデルの設定を行っています）

* 作成したリカレントニューラル言語モデルを導入しています。
* 最適化の手法はRMSpropを使用
http://qiita.com/skitaoka/items/e6afbe238cd69c899b2a
* 初期のパラメータを-0.1〜0.1の間で与えています。


In [21]:
# Prepare RNNLM model
model = CharRNN(len(vocab), n_units)

optimizer = optimizers.RMSprop(lr=2e-3, alpha=0.95, eps=1e-8)
optimizer.setup(model)

## 学習を始める前の設定

* 学習データのサイズを取得 
* ジャンプの幅を設定（順次学習しない）
* パープレキシティを0で初期化 
* 最初の時間情報を取得 
* 初期状態を現在の状態に付与 
* 状態の初期化 
* 損失を0で初期化

In [22]:
whole_len    = train_data.shape[0]
jump         = whole_len / batchsize
epoch        = 0
start_at     = time.time()
cur_at       = start_at
state        = make_initial_state(n_units, batchsize=batchsize)
accum_loss   = Variable(np.zeros((), dtype=np.float32))
cur_log_perp = np.zeros(())

## パラメータ更新方法（ミニバッチ学習）

* ミニバッチを用いて学習している。
* `x_batch = np.array([train_data[(jump * j + i) % whole_len] for j in range(batchsize)])`はデータの一部を連続的に使用せず、飛ばしながら使用することでデータの偏りを防ぐように学習していきます。
* `state, loss_i = model.forward_one_step(x_batch, y_batch, state, dropout_ratio=0.5)`は損失と状態を計算しています。ここで過学習を防ぐdropアウトの率も設定可能です。
* `if (i + 1) % bprop_len == 0`はどれだけ過去の文字を保持するかを表しています。bprop_lenが大きければ大きいほど過去の文字を保持できますが、メモリ破綻を起こす可能性があるのでタスクによって適切な数値に設定する必要があります。
* bprop_lenの詳細について[truncate](http://kiyukuta.github.io/2013/12/09/mlac2013_day9_recurrent_neural_network_language_model.html#recurrent-neural-network)
* `optimizer.clip_grads(grad_clip)`は正則化をかけており、過学習を防いでいます。
* `optimizer.lr *= 0.97`は学習を緩やかにしていく部分です。最初は学習は大きく更新したいですが、だんだんと最適な解に近づいているため緩やかに近づける調整のための工夫です。


In [26]:
for i in range(int(jump * n_epochs)):
    #-------------Explain4 in the Qiita-------------
    x_batch = np.array([train_data[(jump * j + i) % whole_len]
                        for j in range(batchsize)])
    y_batch = np.array([train_data[(jump * j + i + 1) % whole_len]
                        for j in range(batchsize)])

    state, loss_i = model.forward_one_step(x_batch, y_batch, state, dropout_ratio=0.7)
    accum_loss   += loss_i
    cur_log_perp += loss_i.data.reshape(())

    if (i + 1) % bprop_len == 0:  # Run truncated BPTT
        now = time.time()

        print('{}/{}, train_loss = {}, time = {:.2f}'.format((i+1)/bprop_len, jump, accum_loss.data / bprop_len, now-cur_at))
        cur_at = now

        optimizer.zero_grads()
        accum_loss.backward()
        accum_loss.unchain_backward()  # truncate
        accum_loss = Variable(np.zeros((), dtype=np.float32))

        optimizer.clip_grads(grad_clip)
        optimizer.update()
        
    if (i + 1) % 10000 == 0:
        perp = math.exp(cuda.to_cpu(cur_log_perp) / 10000)
        print('iter {} training perplexity: {:.2f} '.format(i + 1, perp))
        fn = ('%s/charrnn_epoch_%.2f.chainermodel' % (checkpoint_dir, float(i)/jump))
        pickle.dump(copy.deepcopy(model).to_cpu(), open(fn, 'wb'))

    if (i + 1) % jump == 0:
        epoch += 1

        if epoch >= 10:
            optimizer.lr *= 0.97
            print('decayed learning rate by a factor {} to {}'.format(0.97, optimizer.lr))
    #-------------Explain4 in the Qiita-------------

    sys.stdout.flush()

1.0/6657.12, train_loss = 9.113505554199218, time = 126.36
2.0/6657.12, train_loss = 9.075080108642577, time = 0.31
3.0/6657.12, train_loss = 9.036331176757812, time = 0.30
4.0/6657.12, train_loss = 8.917592620849609, time = 0.26
5.0/6657.12, train_loss = 8.453337860107421, time = 0.27
6.0/6657.12, train_loss = 7.700492858886719, time = 0.26
7.0/6657.12, train_loss = 7.174608612060547, time = 0.26
8.0/6657.12, train_loss = 6.754265594482422, time = 0.26
9.0/6657.12, train_loss = 6.40767822265625, time = 0.25
10.0/6657.12, train_loss = 6.664650726318359, time = 0.25
11.0/6657.12, train_loss = 6.582872009277343, time = 0.26
12.0/6657.12, train_loss = 6.349908828735352, time = 0.26
13.0/6657.12, train_loss = 6.3104499816894535, time = 0.26
14.0/6657.12, train_loss = 6.1276508331298825, time = 0.26
15.0/6657.12, train_loss = 6.2743183135986325, time = 0.26
16.0/6657.12, train_loss = 6.19116096496582, time = 0.25
17.0/6657.12, train_loss = 6.164059829711914, time = 0.26
18.0/6657.12, train_

/Users/smap6/.pyenv/versions/miniconda3-3.9.1/envs/chainer_env/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/smap6/.pyenv/versions/miniconda3-3.9.1/envs/chainer_env/lib/python3.5/site-packages/ipykernel/__main__.py:6: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


KeyboardInterrupt: 

## 言語の予測

学習時はバイナリでデータを取得していましたが、予測は通常のファイル形式で取得しています。
* `ivocab = {v:k for k, v in vocab.items()}`でインデックスから単語を取得できるように並べ替えをしています。


In [35]:
#-------------Explain5 in the Qiita-------------
def load_predict_data(filename, vocab):
    source = open(filename).read()
    words = source_to_words(source)
    dataset = np.ndarray((len(words),), dtype=np.int32)
    for i, word in enumerate(words):
        if word in vocab:
            dataset[i] = vocab[word]
        else:
            print("{0} is not exist in vocab.".format(word))
    return dataset

predict_data = load_predict_data('data_hands_on/angular_full.js', vocab)

ivocab = {}
ivocab = {v:k for k, v in vocab.items()}
#-------------Explain5 in the Qiita-------------


[   0    1    0 ..., 9008    6   53]


予測では作成されたモデル変更と文字列予測を行ないます。

* モデルを変更する。
* 文字列を予測する。

予測するモデルの変更はここではiPython notebook内の下記のコードを変更します。
作成されたモデルはcvフォルダの中にあるので
あまり数は出来ていませんが、確認して見て下さい。

In [33]:
# load model
#-------------Explain6 in the Qiita-------------
model = pickle.load(open("cv/charrnn_epoch_1.50.chainermodel", 'rb'))
#-------------Explain6 in the Qiita-------------
n_units = model.embed.W.shape[1]

# initialize generator
state = make_initial_state(n_units, batchsize=1, train=False)

prev_char = np.array([0], dtype=np.int32)

* `state, prob = model.predict(prev_char, state)`で予測された確率と状態を取得しています。次の予測にも使用するため状態も取得しています。
* `index = np.argmax(cuda.to_cpu(prob.data))`は`cuda.to_cpu(prob.data)`部分で各単語の重み確率を取得できるため、その中で一番確率が高いものが予測された文字なのでその文字のインデックスを返すようにしています。
* `sys.stdout.write(ivocab[index] + " ")`で予測した文字を出力するための準備です。
* `prev_char = np.array([index], dtype=np.int32)`は次の予測に使用するために過去の文字を保持するのに使用しています。

In [34]:
pattern = re.compile(r".*;.*")
for i in range(1000):
    #-------------Explain7 in the Qiita-------------
    state, prob = model.predict(prev_char, state)
    
    index = np.argmax(cuda.to_cpu(prob.data))

    if ivocab[index] == "{" or ivocab[index] == "}" or pattern.match(ivocab[index]):
        sys.stdout.write("\n")
        split_vocab = ivocab[index].split(";")
        sys.stdout.write(split_vocab[0] + ";")
        sys.stdout.write("\n") 
        if len(split_vocab) >= 2:
            sys.stdout.write(split_vocab[1])
    else:
        sys.stdout.write(ivocab[index] + " ")
        
    prev_char = np.array([index], dtype=np.int32)
    #-------------Explain7 in the Qiita-------------

print

) 
{;

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

};

}

<function print>